# Lesson 3 - Build a Clutter Model

In this lesson you'll create a statistical propagation model to predict loss due to clutter in 3.5 GHz propagation measurements. To create the model you are given two measurement datasets from the Martin Acres neighborhood of Boulder, Colorado. They are located in the `course-materials/data/` directory. 

- **`course-materials/data/MartinAcresLow_3500.MartinAcres.csv`**
- **`course-materials/data/MartinAcresHigh_3500.MartinAcres.csv`**

<img src="./images/measurement_datasets.png" alt="low and high datasets" width="1200"/>

This image shows two datasets made with a 3.5 GHz continuous wave signal transmitted from two different locations: low TX, and high TX. The low TX antenna is at approximately 1660 meters above sea level. The high TX antenna is at approximately 1800 meters above sea level. 

__In this lesson you will:__

**1.** Load the measurement data and plot it on a map.

**2.** Visualize the data: a) Cumulative Distribution and b) Path Distance vs Clutter Loss.

**3.** Determine the _distance within clutter_ for each ray path.

**4.** Fit a regression model to **predict clutter loss based on _distance within clutter_**.

<br/>

### Import the necessary python libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rasterio
from sklearn.linear_model import LinearRegression

### Load Lesson 2 LiDAR clutter statistics for Martin Acres
  * Clutter height mean = 8 meters.
  * Clutter height std. deviation = 4.5 meters.

In [ ]:
clutter_height_mean = 8 ## meters
clutter_height_std = 4.5 ## meters

### Bring in LiDAR data and static variables

In [ ]:
## bring in LiDAR data (same as lesson 2)
dataset_terrain = rasterio.open("./data/MartinAcres.dtm.tif")
dataset_surface = rasterio.open("./data/MartinAcres.dsm.tif")
band1_terrain = dataset_terrain.read(1)
band1_surface = dataset_surface.read(1)
## set map and LiDAR bounds (same as lesson 2)
left, bottom, right, top = dataset_terrain.bounds
left_deg, bottom_deg, right_deg, top_deg = (-105.2869255, 39.9747967, -105.2343079, 40.0019534)
## for converting lat and long degrees into meters (same as lesson 2)
meters_per_lat = 111319.49 
meters_per_long = 85263.24 ## only valid at latitudes of 40 degrees

### Load the measurement datasets

The measurement datasets are loaded from CSV into [Pandas DataFrames](https://www.datacamp.com/tutorial/pandas-tutorial-dataframe-python). DataFrames are a row/column data structure that allow for efficient manipulation and analysis of large amounts of data.

In [ ]:
low_file_name = "./data/MartinAcresLow_3500.MartinAcres.csv"
high_file_name = "./data/MartinAcresHigh_3500.MartinAcres.csv"
low_df = pd.read_csv(low_file_name)
high_df = pd.read_csv(high_file_name)

### Look at the first 5 rows of the high and low DataFrames

In [ ]:
high_df.head(5)

In [ ]:
low_df.head(5)

### Column definitions 

| Column        | Definition                         |
| ------------- |------------------------------------|
| ID            | Identifier                         |
| TxLat         | Tx Latitude                        |
| TxLon         | Tx Longitude                       |
| RxLat         | Rx Latitude                        |
| RxLon         | Rx Longitude                       |
| h_tx__meter   | Tx Antenna Height                  |
| h_rx__meter   | Rx Antenna Height                  |
| f__mhz        | Frequency                          |
| d__km         | Distance TX to RX                  |
| d_3d__km      | 3D Distance TX to RX               |
| L_meas__db    | Measured Loss                      |
| L_fs__db      | Estim. Free-Space Loss             |
| L_excess__db  | Clutter Loss                       |

d_3d__km considers the antenna altitudes when calculating the distance between the TX and RX. Low dataset only has d__km because there is very little  difference from d_3d__km. 

### Add a new columns to the DataFrames

Convert __TxLat, TxLon, RxLat, and RxLon to meters__ for ease of measuring and plotting.

In [ ]:
## functions to convert lat and long to meters
def convert_lat_to_meters(lat):
    lat_dif = top_deg - lat
    vert_dif_m = lat_dif * meters_per_lat
    return vert_dif_m

def convert_long_to_meters(long):
    long_dif = long - left_deg
    horz_dif_m = long_dif * meters_per_long
    return horz_dif_m

## create new columns by mapping the conversion function to lat or long column
low_df["RxX__meter"] = low_df["RxLon"].map(convert_long_to_meters)
low_df["RxY__meter"] = low_df["RxLat"].map(convert_lat_to_meters)
low_df["TxX__meter"] = low_df["TxLon"].map(convert_long_to_meters)
low_df["TxY__meter"] = low_df["TxLat"].map(convert_lat_to_meters)

### Look at the new columns RxX__meter, RxY__meter, TxX__meter, TxY__meter

In [ ]:
low_df.head(5)

In [ ]:
## do the same for the High dataframe
high_df["RxX__meter"] = high_df["RxLon"].map(convert_long_to_meters)
high_df["RxY__meter"] = high_df["RxLat"].map(convert_lat_to_meters)
high_df["TxX__meter"] = high_df["TxLon"].map(convert_long_to_meters)
high_df["TxY__meter"] = high_df["TxLat"].map(convert_lat_to_meters)

In [ ]:
high_df.head(5)

### Plot the measurements on a map

In [ ]:
## read in Martin Acres map image
##  image from www.openstreetmap.org
martin_acres_map = plt.imread('./images/martin_acres_map.png')

## set the map bounding box (in meters)
BBox = (0,  right-left,      
        top-bottom,  0)

In [ ]:
## make the Low plot
plt.rcParams["figure.figsize"] = (10,10)

## plot the TX location
ex2 = plt.scatter(low_df.iloc[0]["TxX__meter"],low_df.iloc[0]["TxY__meter"], zorder=2, alpha=1.0, c='r', s=50, label="Tx Location")
## plot the Clutter Losses 
ex1 = plt.scatter(low_df["RxX__meter"], low_df["RxY__meter"], zorder=1, c=low_df["L_excess__db"], cmap='copper_r', alpha= 1.0, s=6, vmin=0, vmax=40)

cbar = plt.colorbar(ex1, fraction=0.03)
cbar.set_label('Excess Loss (dB)')
plt.title('Clutter Loss for Martin Acres Low Tx Location')
plt.xlabel("Meters")
plt.ylabel("Meters")
plt.xlim(BBox[0],BBox[1])
plt.ylim(BBox[2],BBox[3])
plt.imshow(martin_acres_map, zorder=0, extent = BBox, aspect= 'equal')
plt.legend()
plt.show()

In [ ]:
## make the high plot
plt.rcParams["figure.figsize"] = (10,10)

## plot the Tx location
ex2 = plt.scatter(high_df.iloc[0]["TxX__meter"],high_df.iloc[0]["TxY__meter"], zorder=2, alpha=1.0, c='r', s=50, label="Tx Location")
## plot the Clutter Losses 
ex1 = plt.scatter(high_df["RxX__meter"], high_df["RxY__meter"], zorder=1, c=high_df["L_excess__db"], cmap='copper_r', alpha= 1.0, s=6, vmin=0, vmax=40)

cbar = plt.colorbar(ex1, fraction=0.03)
cbar.set_label('Excess Loss (dB)')
plt.title('Clutter Loss for Martin Acres High Tx Location')
plt.xlabel("Meters")
plt.ylabel("Meters")
plt.xlim(BBox[0],BBox[1])
plt.ylim(BBox[2],BBox[3])
plt.imshow(martin_acres_map, zorder=0, extent = BBox, aspect= 'equal')
plt.legend()
plt.show()

### Plot the Cumulative Distribution Function (CDF) of the Clutter Loss for High and Low

In [ ]:
plt.rcParams["figure.figsize"] = (11,6)

low_N = len(low_df)
low_x = np.sort(low_df["L_excess__db"])
low_y = np.arange(low_N) / float(low_N)
## plot the CDF
plt.plot(low_x, low_y, label='Low TX')

high_N = len(high_df)
high_x = np.sort(high_df["L_excess__db"])
high_y = np.arange(high_N) / float(high_N)
## plot the CDF
plt.plot(high_x, high_y, label='High TX')

plt.xlabel('Clutter Loss (dB)')
plt.ylabel('Probability')
plt.title('Clutter Loss CDF')

plt.gca().yaxis.grid(True)
plt.legend(fontsize=14)
plt.show()

The clutter loss CDF shows that the Low Tx measurements suffers from more clutter loss than the High Tx measurements. 

### Plot the Path Distance vs Clutter Loss

In [ ]:
plt.scatter(low_df["d__km"], low_df["L_excess__db"], label='Low TX', s=12)
plt.scatter(high_df["d__km"], high_df["L_excess__db"], label='High TX', s=12)

plt.xlabel('Distance (m)')
plt.ylabel('Clutter Loss (dB)')
plt.title('Path Distance vs Clutter Loss')

plt.legend(fontsize=14)
plt.gca().yaxis.grid(True)
plt.show()

Clutter loss vs path distance shows there is no obvious global correlation between the two. Instead there seems to be a distinct relationship depending on the transmitter height.

Because of the interesting relationship between transmitter height (high vs low) and the clutter loss. Let's calculate the _Rx elevation angle_ for all of the measurements in both datasets. 

<img src="./images/rx_elev_angle.png" alt="RX elevation angle" width="600"/>

### Add an "_Rx elevation angle_" column to high and low DataFrames

In [ ]:
## use LiDAR to add antenna altitude 
def antenna_altitude(x, y, antenna_height, lidar_elev_band):
    elev = lidar_elev_band[int(y)][int(x)]
    return elev + antenna_height

low_df["alt_rx__meter"] = low_df.apply(lambda row: antenna_altitude(row.RxX__meter, row.RxY__meter, row.h_rx__meter, band1_terrain), axis=1)
low_df["alt_tx__meter"] = low_df.apply(lambda row: antenna_altitude(row.TxX__meter, row.TxY__meter, row.h_tx__meter, band1_terrain), axis=1)
high_df["alt_rx__meter"] = high_df.apply(lambda row: antenna_altitude(row.RxX__meter, row.RxY__meter, row.h_rx__meter, band1_terrain), axis=1)
high_df["alt_tx__meter"] = high_df.apply(lambda row: antenna_altitude(row.TxX__meter, row.TxY__meter, row.h_tx__meter, band1_terrain), axis=1)

## RX elevation angle = arcsin( (tx_altitude - rx_altitude) / Hypotenuse distance )
##  returns Rx elevation angle in degrees
def rx_elev_angle(alt_rx, alt_tx, d):
    soh = np.abs(alt_tx - alt_rx) / (d*1000)
    angle_deg = np.arcsin(soh)*360/(np.pi*2) 
    return angle_deg

high_df["rx_angle__deg"] = high_df.apply(lambda row: rx_elev_angle(row.alt_rx__meter, row.alt_tx__meter, row.d_3d__km), axis=1)
low_df["rx_angle__deg"] = low_df.apply(lambda row: rx_elev_angle(row.alt_rx__meter, row.alt_tx__meter, row.d__km), axis=1)

### Summary Statistics for RX Elevation Angle

In [ ]:
print("\t\t\t\t\tMin\tMax\tMean\tSt Dev")

print("Low - RX elevation angle (deg)\t\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}".format(np.min(low_df["rx_angle__deg"]),
                                                                                     np.max(low_df["rx_angle__deg"]),
                                                                                     np.mean(low_df["rx_angle__deg"]),
                                                                                     np.std(low_df["rx_angle__deg"])))

print("High - RX elevation angle (deg)\t\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}".format(np.min(high_df["rx_angle__deg"]),
                                                                                     np.max(high_df["rx_angle__deg"]),
                                                                                     np.mean(high_df["rx_angle__deg"]),
                                                                                     np.std(high_df["rx_angle__deg"])))

Low has an average Rx elevation angle of 1 degree. High has an average elevation angle of 4 degrees. The difference between 1 and 4 degrees seems to account for an additional 10 dB of clutter loss, according to the CDF plotted earlier.

### Plot Rx elevation angle vs clutter loss

In [ ]:
plt.scatter(low_df["rx_angle__deg"], low_df["L_excess__db"], label='Low TX', s=10)
plt.scatter(high_df["rx_angle__deg"], high_df["L_excess__db"], label='High TX', s=10)

plt.xlabel('Rx Elevation Angle (deg)')
plt.ylabel('Clutter Loss (dB)')
plt.title('Rx Elevation Angle vs Clutter Loss')

plt.legend(fontsize=14)
plt.gca().yaxis.grid(True)
plt.show()

As the previous plots have shown, we see no obvious correlation between the direct ray path distance and the clutter loss. Instead there seems to be some relationship between Rx elevation angle and clutter loss. There are some obvious outliers to this trend i.e., the Low Tx measurements with near 0 clutter loss. We leave it up to you to investigate possible explainations for the outliers.

### Develop the 3D Clutter Distance Model

In this model we assume clutter only appears near the end-point of the receiver. We also assume that west of Martin Acres the clutter ends and there is free space (Broadway acts as the western boundry). Therefore, the clutter will be modeled as a __slab__ resting atop the terrain of the Martin Acres neighborhood. The slab is as tall as the clutter height measured in lesson 2, $h_c$, and is bounded to the west by Broadway. 


<img src="./images/clutter_slab.png" alt="Clutter Slab" width="800"/>

Boardway we'll define as the line that passes through the following two points. You'll plot this line in the next code cell. 
 * bway_north = (39.99741,-105.26273)
 * bway_south = (39.98587,-105.25248)

The clutter height $h_c$ we measured in lesson 2. The mean clutter height $\micro_c$ is 8 meters, and the standard deviation $\sigma_c$ was 4.5 meters. Let's define $h_c$ as: $$h_c = \micro_c + 2\sigma_c$$ $$h_c = \text{17 meters}$$ 
We chose this because this includes about 98% of the clutter. We encourage you to play with difference values of $h_c$ to see how it affects the model and the accuracy of the predictions.

### Calculate 3D Clutter Distance

3D Clutter Distance is the distance the ray traveled through the clutter slab to the RX point within the clutter. There are two scenarios you must consider when finding the 3D Clutter Distance: 

**1.** Find the distance of the path if it exits out of the top of the clutter slab. Defined as $h_c / \sin(\theta_\text{rx})$ where $\theta_\text{rx}$ is the RX elevation angle.

<img src="./images/3d_dist_top.png" alt="RX elevation angle" width="800"/>

**2.** Find the distance of the path if it exits out of the side of the clutter slab, defined as $d_c$. Which can be found by finding the intersection of the ray path with Broadway. 

<img src="./images/3d_dist_side.png" alt="RX elevation angle" width="800"/>

Ultimately, you will find the 3D Clutter Distance, $r_c$, as

$$r_c = \text{MIN} (d_c , \frac{h_c}{\sin\theta_\text{rx}})$$


### Start by plotting the Broadway boundry of the slab

In [ ]:
def convert_gps_to_meters(lat, long):
    lat_dif = top_deg - lat
    long_dif = long - left_deg
    vert_dif_m = lat_dif * meters_per_lat
    horz_dif_m = long_dif * meters_per_long
    return (vert_dif_m, horz_dif_m)

bway_n = convert_gps_to_meters(39.99741,-105.26273)
bway_s = convert_gps_to_meters(39.98587,-105.25248)

## make the plot
plt.rcParams["figure.figsize"] = (10,10)

ex3 = plt.plot([bway_n[1],bway_s[1]], [bway_n[0],bway_s[0]], zorder=3, alpha=1.0, c='b')

plt.title('The Broadway Boundry\n marking the western edge of the slab')
plt.xlabel("Meters")
plt.ylabel("Meters")
plt.xlim(BBox[0],BBox[1])
plt.ylim(BBox[2],BBox[3])
plt.imshow(martin_acres_map, zorder=0, extent = BBox, aspect= 'equal')
plt.show()

### Find the distance from the RX location to the Broadway boundry

and add it as a new column (called "horizontal_d__meter") to the DataFrame.

In [ ]:
## this function finds the intersection of the ray path with broadway
##  it returns the distance of the intersection to the RX point
##  input tx and rx points and Broadway start and end
def m_intersect(tx_m, rx_m, line_s_m, line_e_m):
    rise1 = tx_m[0] - rx_m[0]
    run1 = tx_m[1] - rx_m[1]
    m1 = rise1 / run1 ## find the slope of the first line
    
    rise2 = line_s_m[0] - line_e_m[0]
    run2 = line_s_m[1] - line_e_m[1]
    m2 = rise2 / run2 ## find the slope of the second line
    
    ## x and y are the intersection point (in meters)
    ## x = (m1*b - a - m2*d + c) / (m1 - m2)
    x = (m1*tx_m[1] - tx_m[0] - m2*line_s_m[1] + line_s_m[0]) / (m1 - m2)
    y = m1*(x-tx_m[1]) + tx_m[0]
    # print(m1, m2, x, y)
    dist_from_rx = np.sqrt(np.square(x - rx_m[1]) + np.square(y - rx_m[0]))
    # print(dist_from_rx)
    return dist_from_rx

## add new column to DataFrame, the distance from RX to Broadway
low_df["horizontal_d__meter"] = low_df.apply(lambda row: m_intersect((row.TxY__meter, row.TxX__meter), (row.RxY__meter, row.RxX__meter), bway_s, bway_n), axis=1)
high_df["horizontal_d__meter"] = high_df.apply(lambda row: m_intersect((row.TxY__meter, row.TxX__meter), (row.RxY__meter, row.RxX__meter), bway_s, bway_n), axis=1)

### Plot the distance from RX to Broadway

In [ ]:
## make the plot
plt.rcParams["figure.figsize"] = (10,10)

ex2 = plt.scatter(high_df.iloc[0]["TxX__meter"],high_df.iloc[0]["TxY__meter"], zorder=2, alpha=1.0, c='r', s=50, label="Tx Location")
ex1 = plt.scatter(high_df["RxX__meter"], high_df["RxY__meter"], zorder=1, c=high_df["horizontal_d__meter"], cmap='copper_r', alpha= 1.0, s=4, vmin=0, vmax=1700)
ex3 = plt.plot([bway_n[1],bway_s[1]], [bway_n[0],bway_s[0]], zorder=3, alpha=1.0, c='b')
cbar = plt.colorbar(ex1, fraction=0.03)
cbar.set_label('Meters')

plt.title('Distance from RX point to Broadway Intersection, along TX-RX path\nfor High data')
plt.xlabel("Meters")
plt.ylabel("Meters")
plt.xlim(BBox[0],BBox[1])
plt.ylim(BBox[2],BBox[3])
plt.imshow(martin_acres_map, zorder=0, extent = BBox, aspect= 'equal')
plt.legend()
plt.show()

In [ ]:
## make the plot
plt.rcParams["figure.figsize"] = (10,10)

ex2 = plt.scatter(low_df.iloc[0]["TxX__meter"],low_df.iloc[0]["TxY__meter"], zorder=2, alpha=1.0, c='r', s=50, label="Tx Location")
ex1 = plt.scatter(low_df["RxX__meter"], low_df["RxY__meter"], zorder=1, c=low_df["horizontal_d__meter"], cmap='copper_r', alpha= 1.0, s=4, vmin=0, vmax=1700)
ex3 = plt.plot([bway_n[1],bway_s[1]], [bway_n[0],bway_s[0]], zorder=3, alpha=1.0, c='b')
cbar = plt.colorbar(ex1, fraction=0.03)
cbar.set_label('Meters')

plt.title('Distance from RX point to Broadway Intersection, along TX-RX path\nfor Low data')
plt.xlabel("Meters")
plt.ylabel("Meters")
plt.xlim(BBox[0],BBox[1])
plt.ylim(BBox[2],BBox[3])
plt.imshow(martin_acres_map, zorder=0, extent = BBox, aspect= 'equal')
plt.legend()
plt.show()

### Find the 3D Clutter Distance $r_c$

$r_c = \text{MIN} (d_c , \frac{h_c}{\sin\theta_\text{rx}})$ and add it as a new column called "clutter_d__meter".

In [ ]:
h_c = clutter_height_mean + 2*clutter_height_std

def clutter_distance(horizontal_d, rep_clutter_height, rx_angle):
    return min(horizontal_d, rep_clutter_height/np.sin(rx_angle*(np.pi*2)/360))

## add the new clutter distance columns
low_df["clutter_d__meter"] = low_df.apply(lambda row: clutter_distance(row.horizontal_d__meter, h_c, row.rx_angle__deg), axis=1)
high_df["clutter_d__meter"] = high_df.apply(lambda row: clutter_distance(row.horizontal_d__meter, h_c, row.rx_angle__deg), axis=1)

$L_c = L_{c,m} + Y(p)$ clutter loss is the median clutter loss + the location variability modeled as a log normal distrobution. $p$ is the percentage of locations.

$L_{c,m} = a\text{log}_{10}r_c+b$

$r_c = \text{MIN}(d_c,\frac{h_c}{\text{sin}\theta})$

### plot the 3D Clutter Distance vs Clutter Loss

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)
plt.scatter(low_df["clutter_d__meter"], low_df["L_excess__db"], label='Low TX', s=8, zorder=1)
plt.scatter(high_df["clutter_d__meter"], high_df["L_excess__db"], label='High TX', s=8, zorder=0)

plt.xlabel('3D Clutter Distance (m)')
plt.ylabel('Clutter Loss (dB)')
plt.title('3D Clutter Distance vs Clutter Loss')
  
plt.legend(fontsize=14)
plt.show()

### Plot the logarithmic transform of the data

Apply a log transform to the x axis.

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)

plt.scatter(np.log10(low_df["clutter_d__meter"]), low_df["L_excess__db"], label='Low TX', s=8, zorder=1)
plt.scatter(np.log10(high_df["clutter_d__meter"]), high_df["L_excess__db"], label='High TX', s=8, zorder=0)


plt.xlabel('Log10(3D Clutter Distance)')
plt.ylabel('Clutter Loss (dB)')
plt.title('Logarithmic transform of the data')
  
plt.legend(fontsize=14)
plt.show()

### Fit data with a linear regression line

In [ ]:
## concatinate the high and low dataframes
df1 = high_df[["clutter_d__meter", "L_excess__db"]]
df2 = low_df[["clutter_d__meter", "L_excess__db"]]
frames = [df1, df2]
full_df = pd.concat(frames)

## perform the linear regression on the data
linear_regression = LinearRegression()
linear_regression.fit(np.array(np.log10(full_df["clutter_d__meter"])).reshape(-1,1), full_df["L_excess__db"])
y_int = linear_regression.intercept_
slope = linear_regression.coef_[0]
print("y = {:.2f} * log10(x) + {:.2f}".format(slope, y_int))

### Plot with the regression line

In [ ]:
plt.scatter(np.log10(low_df["clutter_d__meter"]), low_df["L_excess__db"], label='Low TX', s=8, zorder=1)
plt.scatter(np.log10(high_df["clutter_d__meter"]), high_df["L_excess__db"], label='High TX', s=8, zorder=0)

## add the regression line
z_point = (1.5, slope*1.5 + y_int)
n_point = (3.3, slope*3.3 + y_int)

plt.plot([z_point[0], n_point[0]],[z_point[1], n_point[1]], c='b', label="Linear Model")
plt.xlabel('Log10(3D Clutter Distance)')
plt.ylabel('Clutter Loss (dB)')
plt.title('Linear Regression on data')
  
plt.legend(fontsize=16)
plt.show()

### Transform back to linear units

In [ ]:
full_x = np.linspace(5,1750,2000)
full_y = slope*np.log10(full_x) + y_int
plt.plot(full_x, full_y, label='Model', c='b')

plt.scatter(low_df["clutter_d__meter"], low_df["L_excess__db"], label='Low TX', s=8, zorder=1)
plt.scatter(high_df["clutter_d__meter"], high_df["L_excess__db"], label='High TX', s=8, zorder=0)

plt.xlabel('3D Clutter Distance (m)', fontsize=15)
plt.ylabel('Clutter Loss (dB)', fontsize=15)
plt.title('Fig. 7\nLog 3D Clutter Distance Model', fontsize=16)
plt.grid()
plt.legend(fontsize=14)
plt.show()

### Your median clutter loss model is...

In [ ]:
print("y = {:.2f} * log10(x) + {:.2f}".format(slope, y_int))

In [ ]:
def model(r_c):
    return slope * np.log10(r_c) + y_int

# df1 = high_df[["clutter_d__meter", "L_excess__db"]]
# df2 = low_df[["clutter_d__meter", "L_excess__db"]]
# frames = [df1, df2]
# full_df = pd.concat(frames)


full_df["pred_loss"] = full_df["clutter_d__meter"].map(model)
# low_df["alt_rx__meter"] = low_df.apply(lambda row: antenna_altitude(row.RxX__meter, row.RxY__meter, row.h_rx__meter, band1_terrain), axis=1)
full_df["error"] = full_df.apply(lambda row: row.L_excess__db - row.pred_loss, axis=1)

In [ ]:
full_x = np.linspace(5,1750,2000)
full_y = slope*np.log10(full_x) + y_int
plt.plot(full_x, full_y, label='Model', c='b')

plt.scatter(full_df["clutter_d__meter"], full_df["pred_loss"], label='Predicted', s=8, zorder=1)

plt.xlabel('3D Clutter Distance (m)', fontsize=15)
plt.ylabel('Clutter Loss (dB)', fontsize=15)
plt.title('Fig. 7\nLog 3D Clutter Distance Model', fontsize=16)
plt.grid()
plt.legend(fontsize=14)
plt.show()

In [ ]:
plt.scatter(full_df["L_excess__db"], full_df["pred_loss"], label='Predicted', s=8, zorder=1)

plt.xlabel('actual', fontsize=15)
plt.ylabel('predicted', fontsize=15)
plt.title('Fig. 7\nLog 3D Clutter Distance Model', fontsize=16)
plt.grid()
plt.legend(fontsize=14)
plt.show()

In [ ]:
x = np.linspace(0,1000,len(full_df["L_excess__db"]))
plt.scatter(x, full_df["error"], label='error', s=8, zorder=1)

plt.xlabel('meas', fontsize=15)
plt.ylabel('error', fontsize=15)
plt.title('Fig. 7\nLog 3D Clutter Distance Model', fontsize=16)
plt.grid()
plt.legend(fontsize=14)
plt.show()

In [ ]:
RMSE = np.sqrt( np.sum(np.square(full_df["error"]))/len(full_df["error"]) )
print(RMSE)

MAE = np.mean(abs(full_df["error"]))
print(MAE)

In [ ]:
full_x = np.linspace(5,1750,2000)
full_y = slope*np.log10(full_x) + y_int
plt.plot(full_x, full_y, label="Our Model", c='b')

b_y = 14.6*np.log10(full_x) - 12.289
plt.plot(full_x, b_y, label="EuCAP Model", c='g')

plt.scatter(high_df["clutter_d__m"], high_df["L_excess__db"], label='High TX', s=8)
plt.scatter(low_df["clutter_d__m"], low_df["L_excess__db"], label='Low TX', s=8)
# plt.scatter(low_rc, low_clutter_loss, label='Low TX', s=12)
# plt.scatter(high_rc, high_clutter_loss, label='High TX', s=12)

plt.xlabel('3D Clutter Distance (m)', fontsize=15)
plt.ylabel('Clutter Loss (dB)', fontsize=15)
plt.title('Fig. 7\nLog 3D Clutter Distance Model', fontsize=16)
plt.grid()
plt.legend(fontsize=14)
plt.show()

# REQUIREMENTS: Calculation vs Estimation
## Calculation
* Measurements
* Calc. $\mu_c$ Mean Clutter Height, REQUIRES: DTM, DSM, (Lat, Long) of measurement area.
* Calc. $\theta$ RX angle, REQUIRES: DTM, (Lat, Long) of RXs, (Lat, Long) of TX.
* Calc. $d_c$ horizontal distance through clutter, REQUIRES: (Lat, Long) of RXs, (Lat, Long) of TX, (Lat, Long) endpoints of the clutter boundry.
## Estimation
* Measurements
* Estimate $\mu_c$ Mean Clutter Height.
* Calc. $\theta$ RX angle, REQUIRES: DTM, (Lat, Long) of RXs, (Lat, Long) of TX.
* Calc. $d_c$ horizontal distance through clutter, REQUIRES: (Lat, Long) of RXs, (Lat, Long) of TX, (Lat, Long) endpoints of the clutter boundry.


### $y = a \text{log}_{10}(x) + b$
$a$ and $b$ are found through regression with measurements, $\mu_c$, $\theta$s, and $d_c$s.

### Max's $y = 13.4\text{log}_{10}(x) - 8.57$
### Billy's $y = 14.6\text{log}_{10}(x) - 12.289$

How do you find $a$ and $b$ without measurements?

How is frequency factored in?

In [ ]:
print(np.min(low_df["d__km"]), np.max(low_df["d__km"]), np.mean(low_df["d__km"]), np.std(low_df["d__km"]))
print(np.min(high_df["d__km"]), np.max(high_df["d__km"]), np.mean(high_df["d__km"]), np.std(high_df["d__km"]))